In [2]:
// Sempre que precisamos de fazer Console.ReadLine num jupiter notebook precisamos de correr esta célula uma vez.
// Não apagues pois podemos vir a precisar dela. OLAAAAAAAAAAAAA
static class Console {
    public static void WriteLine(object o) => System.Console.WriteLine(o);
    public static void WriteLine(bool b) => System.Console.WriteLine(b);
    public static void WriteLine(string text) => System.Console.WriteLine(text);
    public static void WriteLine(int n) => System.Console.WriteLine(n);
    public static string ReadLine() => GetInputAsync("").GetAwaiter().GetResult();
}

# Sistema de Gestão de Residência Universitária
**Projeto POO 2025 – Parte Inicial**

**1. Descrição Geral**
O objetivo é desenvolver uma aplicação para gerir o alojamento de estudantes numa residência.
O sistema deve controlar a ocupação dos diferentes tipos de quartos e manter o registo
atualizado dos residentes e da sua condição financeira/social.

**2. Hierarquia de Classes e Requisitos**
Alojamento (Quartos)
Deve implementar uma estrutura de classes para representar os quartos, utilizando o conceito
de herança:
-  **Alojamento:** Contém atributos comuns como número do quarto, área base e preço base,
mas não pode ser instanciado.
-  **Quarto Partilhado:** Define o número de camas/lugares (entre 2 e 4). O preço base é
dividido pelos lugares. Há um desconto de 10% por cada lugar adicional (aplicável apenas
a 3 (10%) e 4 lugares (20%)).
- **Quarto Individual:** Quarto para apenas um ocupante. Tem um custo adicional de 10%
face ao preço base do quarto.
- **Quarto Individual Premium:** Herda de Quarto Individual, mas permite guardar uma área
superior. Além do custo de quarto individual, paga mais 4% do valor do quarto, por cada
m2 extra de área.
Estudantes e Apoio Social
A gestão de utilizadores deve distinguir os estudantes com base nas suas condições económicas:
- **Estudante:** Nome, NIF, Instituição de Ensino, Rendimento Mensal do Agregado e
Dimensão do Agregado Familiar.
- **Estudante com Apoio Social:** Inclui um Escalão de Apoio e a taxa de desconto na
mensalidade.

**Regra de Negócio (Cálculo do Escalão):**

O escalão é atribuído automaticamente com base no Rendimento per Capita (Rpc):
Rpc = Rendimento do Agregado / Dimensão do Agregado
- Escalão A: Rpc < 300€ - Desconto máximo de 66%
- Escalão B: 300€ <= Rpc < 450€ - Desconto Máximo de 50%
- Escalão C: 450€ <= Rpc <= 600€ - Desconto Máximo de 35%
- Não Carenciado: Rpc > 600€ (Deve ser tratado como Estudante comum).

A taxa de desconto depende do alojamento atribuído. Quando afeto a um alojamento, esta
taxa deve ser atualizada, sendo que o estudante apenas pode pagar 50% do seu Rpc. A
diferença entre o valor do alojamento e o valor pago, deve ser usada para calcular a taxa de
desconto.

**3. Interfaces**
Devem ser implementadas as seguintes Interfaces:

***ICalculavel (implementada pelos alojamentos):***
- double Mensalidade() – Devolve o valor da mensalidade por ocupante;
- double Mensalidade(Estudante ocupante) – Devolve o valor da mensalidade para o ocupante
(atribui eventual desconto na m§ensalidade para estudantes com Apoio Social);
- double Ocupacao() – devolve a taxa de ocupação do alojamento (entre 0 e 1).

***IRendimento (implementada pelos estudantes):***
- double RendimentoPerCapita() – devolve o rendimento do estudante, considerando o valor e
dimensão (ver fórmula acima).

**4. Aplicação**
 – Deve ser criada uma “Residência” com 20 alojamentos
Cada grupo define a proporção quantidade existente de cada tipologia de alojamentos, mas
todos devem existir.
Cada grupo deve ainda definir como associar os estudantes aos alojamentos. Mas deve ser
possível perceber onde está cada estudante alojado, e quem está alojado em cada quarto/lugar.
A aplicação deve criar os alojamentos e depois ter uma interface que permita registar novos
estudantes (pedindo os dados ao utilizador), qualquer que seja o tipo, e permitir a escolha
atribuição de quarto, de acordo com a tipologia e disponibilidade.
Na atribuição de alojamentos a estudantes carenciados, a aplicação deve impedir que esta
aconteça se o valor absoluto do “desconto” for superior ao Rendimento per Capita do estudante.

**5. Exceções**


5. As Exceções
Devem ser tratadas as situações de eventuais erros, capturando e lançando exceções.
------------------------------------------------

**Funcionalidades Esperadas**

1. Registo e Listagem: Adicionar novos quartos e estudantes ao sistema.
2. Atribuição de Alojamento: Validar se um quarto partilhado ainda tem vagas antes de
alocar um estudante. E se os individuais estão livres.
3. Cálculo de Mensalidade: Definido na Interface ICalculavel.
4. Estatísticas: Listar total de rendimento mensal da residência e percentagem de
ocupação.

In [12]:
public class QuartoCheioException : Exception{
  //msg vai buscar a mensgem da parte asseguir
  public QuartoCheioException(string msg) : base(msg){}
}
 
  interface ICalculavel {
    double Mensalidade();
    double Ocupacao();
    double Mensalidade(Estudante ocupante);

    
}

 interface IRendimento {
    double RendimentoPerCapita();
    
}
 
 public abstract class Alojamento : ICalculavel {
    public int NumeroQuarto {get;set;}
    public double AreaBase {get;set;}
    public double PrecoBase {get;private set;} 
    public abstract int Capacidade();
    int ocupantes = 0;

    protected List<Estudante> Residentes = new List <Estudante>();


    public void AdicionarEstudantes(Estudante estudante)
    {
      if (Residentes.Count >= Capacidade())
      {
        throw new QuartoCheioException("");

      }
      Residentes.Add(estudante);
    }
      
      public virtual double Mensalidade(){

        return PrecoBase;
      }


      public double Mensalidade(Estudante ocupante)
    {
        double valorQuarto = Mensalidade(); 

        if (ocupante is EstudanteApoioSocial social)
        {
            double rpc = social.RendimentoPerCapita();
            
            double valorMaximoQuePodePagar = rpc * 0.50;

            if (valorQuarto > valorMaximoQuePodePagar)
            {
                return valorMaximoQuePodePagar;
            }
        }
        return valorQuarto;
    }

      public double Ocupacao()
    {
      //taxa de ocupação  = ocupantes /capacidade 
        return (double)ocupantes / Capacidade();
    }

// para nao ter de repetir o Console.WriteLine(residencia[0].NumeroQuarto);
  public string Identificar(){
    return $"Número Quarto: {NumeroQuarto}, Area Base: {AreaBase}, Preço Base: {PrecoBase}";


  }
    public Alojamento(int numeroQuarto, double areaBase, double precoBase)
    {
        NumeroQuarto = numeroQuarto;
        AreaBase = areaBase;
        PrecoBase = precoBase;
    }     
 }


 public class QuartoPartilhado : Alojamento {
   public int NumeroCamas {get;set;}
    public override int Capacidade() => NumeroCamas;

    public QuartoPartilhado(int numerocamas, int numeroQuarto, double areaBase, double precoBase)
        : base(numeroQuarto, areaBase, precoBase)
   {  
      this.NumeroCamas = numerocamas;

      if (numerocamas > 4 || numerocamas <2 ){
         //fazer depois o try catch
         throw new ArgumentException ("Tem de ser entre 2 e 4");
         //throw new ValorInvalidoException("Saldo tem de ser entre 2 4 ");

      }
   }

      public override double Mensalidade(){

          double precoCama = PrecoBase / Capacidade();

         if (Capacidade() is 4){
            return precoCama * (0.80);
         }
         else if ( Capacidade() is 3){
            return precoCama * (0.90);

         }
         //caso sejam 2 camas
         return precoCama;
      }
   }
 


 
 public class QuartoIndividual : Alojamento {
   
   public override int Capacidade() => 1;
     public QuartoIndividual(int numeroQuarto, double areaBase, double precoBase)
        : base(numeroQuarto, areaBase, precoBase)
    {
    }
    // não ´re preciso devidir por Capacidade, pois vai ser 1
       public override double Mensalidade(){
         return PrecoBase + (PrecoBase * 0.10);
         
      }
 }
 
  public class QuartoPremium : QuartoIndividual{
      public double AreaExtra {get; set;}

        public QuartoPremium(int numeroQuarto, double areaBase, double precoBase, double areaExtra)
        : base(numeroQuarto, areaBase, precoBase)
        {
         AreaExtra = areaExtra;
        }

        public override double Mensalidade(){
         // buscar Preço base do QuartoIndividual;
            double PrecoBaseQuartoIndividual = base.Mensalidade();
         // paga mais 4% do valor do quarto, por cada m2 extra de área.
            return PrecoBaseQuartoIndividual + (PrecoBaseQuartoIndividual * AreaExtra * 0.04) ;

        }
 }



// Rpc = Rendimento do Agregado / Dimensão do Agregado
//Nome, NIF, Instituição de Ensino, Rendimento Mensal do Agregado e Dimensão do Agregado Familiar.
   public class Estudante : IRendimento{
      public string Nome {get; set;}
      public int NIF {get; set;}
      public string  InstituicaodeEnsino {get; set;}
      public double RendimentoAgregado {get;set;}
      public int DimensaoAgregadoFamiliar {get;set;}



      public Estudante (string nome, int nIF, string instituicaodeEnsino, double rendimento,int dimensaoAgregadoFamiliar)
      {
         Nome = nome;
         NIF = nIF;
         InstituicaodeEnsino = instituicaodeEnsino;
         RendimentoAgregado = rendimento;
         DimensaoAgregadoFamiliar = dimensaoAgregadoFamiliar;
      }

      // Rpc = Rendimento do Agregado / Dimensão do Agregado

      public double RendimentoPerCapita()
      {
          if (DimensaoAgregadoFamiliar < 1 ){
         throw new ArgumentException ("Não podes ter um Agregado familiar com 0 ou menos pessoas");
      }
      return RendimentoAgregado / DimensaoAgregadoFamiliar;
      }
      
      }




public class EstudanteApoioSocial: Estudante{
    public string Escalao {get;set;}
    public double TaxaDesconto {get;set;}

public EstudanteApoioSocial (string nome, int nIF, string instiuicaoDeEnsino, double rendimentoMensalAgregado, int dimensaoAgregadoFamiliar)
: base (nome,nIF,instiuicaoDeEnsino,rendimentoMensalAgregado,dimensaoAgregadoFamiliar)


{
    CalcularApoioSocial();
}


public void CalcularApoioSocial()
{
    double rpc = RendimentoPerCapita(); 

    if (rpc < 300){
        Escalao = "A";
        TaxaDesconto = 0.66;
    }
    else if (300 >= rpc && rpc < 450){
        Escalao = "B";
        TaxaDesconto = 0.50; 
    }

     else if (450 >= rpc && rpc <= 600){
        Escalao = "C";
        TaxaDesconto = 0.35; 
    }
    else if (rpc > 600 ){
        Escalao = "NaoCarenciado";
        TaxaDesconto = 0; 
    }
}
}
 


try {

List<Alojamento> residencia = new List<Alojamento>();
residencia.Add( new QuartoIndividual(102, 300, 2));
residencia.Add( new QuartoPartilhado(104, 300, 2, 4));


foreach (var quarto in residencia){
  Console.WriteLine(quarto.Identificar());



}



}

catch{

}

 // adicionar try catch para RendimentoPerCapita | | throw new ArgumentException ("Não podes ter um Agregado familiar com 0 ou menos pessoas");
 // adicionar try catch para  QuartoPartilhado | | throw new ArgumentException ("Tem de ser entre 2 e 4");




Número Quarto: 102, Area Base: 300, Preço Base: 2
